In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
from math import *
import time

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [4]:
# defination of activation function
def activation(x):
    return x * torch.sigmoid(x)

In [5]:
# Build ResNet with three blocks
class net(torch.nn.Module):
    def __init__(self,input_width,layer_width):
        super(net,self).__init__()
        self.layer_in = torch.nn.Linear(input_width, layer_width)
        self.layer1 = torch.nn.Linear(layer_width, layer_width)
        self.layer2 = torch.nn.Linear(layer_width, layer_width)
        self.layer3 = torch.nn.Linear(layer_width, layer_width)
        self.layer4 = torch.nn.Linear(layer_width, layer_width)
        self.layer5 = torch.nn.Linear(layer_width, layer_width)
        self.layer6 = torch.nn.Linear(layer_width, layer_width)
        self.layer_out = torch.nn.Linear(layer_width, 1)
    def forward(self,x):
        y = self.layer_in(x)
        y = y + activation(self.layer2(activation(self.layer1(y)))) # residual block 1
        y = y + activation(self.layer4(activation(self.layer3(y)))) # residual block 2
        y = y + activation(self.layer6(activation(self.layer5(y)))) # residual block 3
        output = self.layer_out(y)
        return output

In [6]:
dimension = 8

In [7]:
# Defination of exact solution
def u_ex(x):  
    x_norm = torch.norm(x, dim = 1) # compute norm of x; dim = 1 - > sum by row
    u_x = torch.sin(pi/2*(1-x_norm)).reshape([x.size()[0],1])
    return u_x

In [8]:
# Defination of f(x)
def f(x):
    x_norm = torch.norm(x, dim = 1)
    u_temp = u_ex(x)
    f_temp = 1/4*pi**2*torch.sin(pi/2*(1-x_norm)) + 1/2*pi*(dimension-1)/x_norm*torch.cos(pi/2*(1-x_norm))  
    return f_temp.reshape([x.size()[0],1]) + u_temp**3

In [9]:
# Generate points by random
data_size = 1000
def generate_data():
    i = 0
    x = torch.tensor([])
    while x.size()[0] < data_size:
        temp_x = 2*torch.rand(data_size, dimension) - 1
        index = torch.where(torch.norm(temp_x, dim = 1) < 1)
        temp_x = temp_x[index]
        x = torch.cat((x,temp_x),0)
    x = x[1:data_size, :]
    return x.cuda()

In [10]:
# Defination of loss function for DRM
def loss_function(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) 
    f_temp = f(x)
    part_1 = torch.sum(1/2*uxsq + 1/4*u_hat**4 - f_temp*u_hat)/x.size()[0]
    
    Nb = 200
    y = (2*torch.rand(Nb, dimension)-1).cuda()
    y_norm = torch.norm(y, dim = 1).cuda()
    for i in range(dimension):
        y[:, i] = y[:, i] / y_norm
    part_2 = (torch.sum((model(y)-u_ex(y))**2)/y.size()[0])
    
    lambda1 = 400.0
    return part_1 + lambda1 * part_2 

In [11]:
# Parameters initialization by normal distribution for neural networks
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m,nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = net(dimension, 16)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

net(
  (layer_in): Linear(in_features=8, out_features=16, bias=True)
  (layer1): Linear(in_features=16, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=16, bias=True)
  (layer3): Linear(in_features=16, out_features=16, bias=True)
  (layer4): Linear(in_features=16, out_features=16, bias=True)
  (layer5): Linear(in_features=16, out_features=16, bias=True)
  (layer6): Linear(in_features=16, out_features=16, bias=True)
  (layer_out): Linear(in_features=16, out_features=1, bias=True)
)

In [14]:
# Defination of relative error
def relative_error():
    x = generate_data()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [15]:
traintime = 40000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [16]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = generate_data()
    x.requires_grad = True
    loss = loss_function(x)
    loss.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", loss.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_8D_Dirichlet_nonlinear_swish.npy", error_save)
np.save("DRM_relative_error_8D_Dirichlet_nonlinear_swish.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(15818.7545, device='cuda:0')
current relative error is:  tensor(20.6241, device='cuda:0')
current epoch is:  50
current loss is:  tensor(343.0719, device='cuda:0')
current relative error is:  tensor(4.0158, device='cuda:0')
current epoch is:  100
current loss is:  tensor(157.2650, device='cuda:0')
current relative error is:  tensor(2.8801, device='cuda:0')
current epoch is:  150
current loss is:  tensor(91.8950, device='cuda:0')
current relative error is:  tensor(2.5038, device='cuda:0')
current epoch is:  200
current loss is:  tensor(90.6088, device='cuda:0')
current relative error is:  tensor(2.2768, device='cuda:0')
current epoch is:  250
current loss is:  tensor(75.7189, device='cuda:0')
current relative error is:  tensor(1.9671, device='cuda:0')
current epoch is:  300
current loss is:  tensor(40.4398, device='cuda:0')
current relative error is:  tensor(1.8315, device='cuda:0')
current epoch is:  350
current loss is:  tensor(32.7412, de

current epoch is:  3100
current loss is:  tensor(0.9443, device='cuda:0')
current relative error is:  tensor(0.9265, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(0.7273, device='cuda:0')
current relative error is:  tensor(0.9222, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(0.8567, device='cuda:0')
current relative error is:  tensor(0.9239, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(0.7928, device='cuda:0')
current relative error is:  tensor(0.9236, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(0.9432, device='cuda:0')
current relative error is:  tensor(0.9286, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(0.7740, device='cuda:0')
current relative error is:  tensor(0.9203, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(0.6355, device='cuda:0')
current relative error is:  tensor(0.9358, device='cuda:0')
current epoch is:  3450
current loss is:  tensor(0.8367, devic

current epoch is:  6200
current loss is:  tensor(-0.0712, device='cuda:0')
current relative error is:  tensor(0.8951, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(-0.0662, device='cuda:0')
current relative error is:  tensor(0.8854, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(-0.0663, device='cuda:0')
current relative error is:  tensor(0.8966, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(-0.0997, device='cuda:0')
current relative error is:  tensor(0.8887, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(-0.0731, device='cuda:0')
current relative error is:  tensor(0.8936, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(-0.0304, device='cuda:0')
current relative error is:  tensor(0.8897, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(-0.0577, device='cuda:0')
current relative error is:  tensor(0.8994, device='cuda:0')
current epoch is:  6550
current loss is:  tensor(-0.104

current epoch is:  9250
current loss is:  tensor(-0.3258, device='cuda:0')
current relative error is:  tensor(0.7745, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(-0.3137, device='cuda:0')
current relative error is:  tensor(0.7695, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(-0.3560, device='cuda:0')
current relative error is:  tensor(0.7662, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(-0.3380, device='cuda:0')
current relative error is:  tensor(0.7556, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(-0.4417, device='cuda:0')
current relative error is:  tensor(0.7323, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(-0.3937, device='cuda:0')
current relative error is:  tensor(0.7204, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(-0.4068, device='cuda:0')
current relative error is:  tensor(0.7218, device='cuda:0')
current epoch is:  9600
current loss is:  tensor(-0.452

current epoch is:  12300
current loss is:  tensor(-1.2412, device='cuda:0')
current relative error is:  tensor(0.1195, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(-1.3976, device='cuda:0')
current relative error is:  tensor(0.1343, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(-1.2566, device='cuda:0')
current relative error is:  tensor(0.1181, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(-1.3129, device='cuda:0')
current relative error is:  tensor(0.1118, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(-1.3387, device='cuda:0')
current relative error is:  tensor(0.1178, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(-1.1376, device='cuda:0')
current relative error is:  tensor(0.1156, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(-1.2282, device='cuda:0')
current relative error is:  tensor(0.1131, device='cuda:0')
current epoch is:  12650
current loss is:  tenso

...

current epoch is:  27550
current loss is:  tensor(-1.3045, device='cuda:0')
current relative error is:  tensor(0.1845, device='cuda:0')
current epoch is:  27600
current loss is:  tensor(-1.4159, device='cuda:0')
current relative error is:  tensor(0.2144, device='cuda:0')
current epoch is:  27650
current loss is:  tensor(-1.4123, device='cuda:0')
current relative error is:  tensor(0.1761, device='cuda:0')
current epoch is:  27700
current loss is:  tensor(-1.4541, device='cuda:0')
current relative error is:  tensor(0.1722, device='cuda:0')
current epoch is:  27750
current loss is:  tensor(-1.4567, device='cuda:0')
current relative error is:  tensor(0.1663, device='cuda:0')
current epoch is:  27800
current loss is:  tensor(-1.2759, device='cuda:0')
current relative error is:  tensor(0.1746, device='cuda:0')
current epoch is:  27850
current loss is:  tensor(-1.4610, device='cuda:0')
current relative error is:  tensor(0.1729, device='cuda:0')
current epoch is:  27900
current loss is:  tenso

current epoch is:  30600
current loss is:  tensor(-1.3305, device='cuda:0')
current relative error is:  tensor(0.1550, device='cuda:0')
current epoch is:  30650
current loss is:  tensor(-1.2714, device='cuda:0')
current relative error is:  tensor(0.1845, device='cuda:0')
current epoch is:  30700
current loss is:  tensor(-1.4165, device='cuda:0')
current relative error is:  tensor(0.2067, device='cuda:0')
current epoch is:  30750
current loss is:  tensor(-1.5567, device='cuda:0')
current relative error is:  tensor(0.1803, device='cuda:0')
current epoch is:  30800
current loss is:  tensor(-1.3217, device='cuda:0')
current relative error is:  tensor(0.1480, device='cuda:0')
current epoch is:  30850
current loss is:  tensor(-1.5254, device='cuda:0')
current relative error is:  tensor(0.1682, device='cuda:0')
current epoch is:  30900
current loss is:  tensor(-1.3176, device='cuda:0')
current relative error is:  tensor(0.1903, device='cuda:0')
current epoch is:  30950
current loss is:  tenso

current epoch is:  33650
current loss is:  tensor(-1.4507, device='cuda:0')
current relative error is:  tensor(0.1767, device='cuda:0')
current epoch is:  33700
current loss is:  tensor(-1.3439, device='cuda:0')
current relative error is:  tensor(0.1888, device='cuda:0')
current epoch is:  33750
current loss is:  tensor(-1.4358, device='cuda:0')
current relative error is:  tensor(0.1887, device='cuda:0')
current epoch is:  33800
current loss is:  tensor(-1.3969, device='cuda:0')
current relative error is:  tensor(0.2045, device='cuda:0')
current epoch is:  33850
current loss is:  tensor(-1.4145, device='cuda:0')
current relative error is:  tensor(0.1943, device='cuda:0')
current epoch is:  33900
current loss is:  tensor(-1.3939, device='cuda:0')
current relative error is:  tensor(0.1656, device='cuda:0')
current epoch is:  33950
current loss is:  tensor(-1.4226, device='cuda:0')
current relative error is:  tensor(0.1496, device='cuda:0')
current epoch is:  34000
current loss is:  tenso

current epoch is:  36700
current loss is:  tensor(-1.4331, device='cuda:0')
current relative error is:  tensor(0.1752, device='cuda:0')
current epoch is:  36750
current loss is:  tensor(-1.3810, device='cuda:0')
current relative error is:  tensor(0.1740, device='cuda:0')
current epoch is:  36800
current loss is:  tensor(-1.4352, device='cuda:0')
current relative error is:  tensor(0.1915, device='cuda:0')
current epoch is:  36850
current loss is:  tensor(-1.2891, device='cuda:0')
current relative error is:  tensor(0.1698, device='cuda:0')
current epoch is:  36900
current loss is:  tensor(-1.4457, device='cuda:0')
current relative error is:  tensor(0.1898, device='cuda:0')
current epoch is:  36950
current loss is:  tensor(-1.4003, device='cuda:0')
current relative error is:  tensor(0.1852, device='cuda:0')
current epoch is:  37000
current loss is:  tensor(-1.3590, device='cuda:0')
current relative error is:  tensor(0.1881, device='cuda:0')
current epoch is:  37050
current loss is:  tenso

current epoch is:  39750
current loss is:  tensor(-1.4489, device='cuda:0')
current relative error is:  tensor(0.1698, device='cuda:0')
current epoch is:  39800
current loss is:  tensor(-1.4287, device='cuda:0')
current relative error is:  tensor(0.1729, device='cuda:0')
current epoch is:  39850
current loss is:  tensor(-1.4801, device='cuda:0')
current relative error is:  tensor(0.1825, device='cuda:0')
current epoch is:  39900
current loss is:  tensor(-1.3272, device='cuda:0')
current relative error is:  tensor(0.2057, device='cuda:0')
current epoch is:  39950
current loss is:  tensor(-1.3768, device='cuda:0')
current relative error is:  tensor(0.1824, device='cuda:0')
total time is:  5829.969738960266 seconds
